# Data Cleaning (w/ some Feature Engineering)

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import spacy
import nltk

## Read-In Data

In [2]:
subreddits = pd.read_csv('../data/subreddits.csv')

In [3]:
subreddits.head(2)

,Unnamed: 0,title,selftext,subreddit,created_utc,author,num_comments,score,is_self,timestamp
0,0,LPT: Answers to why,[removed],LifeProTips,1595040153,AlienAgency,2,1,True,2020-07-17
1,1,¿Quieres obtener juegos y premios gratis en tu...,[removed],LifeProTips,1595040616,GarbageMiserable0x0,2,1,True,2020-07-17


## Data Cleaning

### Drop Unnecessary Columns

#### Drop 'Unnamed: 0'

In [4]:
subreddits.drop(columns = 'Unnamed: 0', inplace = True)

#### Drop 'created_utc' column and 'is_self' columns

In [5]:
subreddits.drop(columns = ['created_utc', 'is_self'], inplace = True)

### Clean Title Column

#### Remove 'UPLT Requests' 

In [6]:
for index, title in enumerate(subreddits['title']):
    if 'ulpt request' in title.lower():
        subreddits.drop(index, inplace = True)

#### Remove 'LPT' and ULPT' from title

In [7]:
for title in subreddits['title']:
    if 'LPT' in title or 'ULPT' in title:
        subreddits['title'] = subreddits['title'].str.replace('ULPT:', '')
        subreddits['title'] = subreddits['title'].str.replace('LPT:', '')
        

### Clean 'SelfText' Column

In [8]:
subreddits['selftext'].fillna('', inplace = True)
subreddits['selftext'] = subreddits['selftext'].str.replace('\[removed\]', '')
subreddits['selftext'] = subreddits['selftext'].str.replace('\[\]', '')

### Clean 'Num_Comments' Column

In [9]:
subreddits['num_comments'].isna().sum()

0

In [10]:
subreddits['num_comments'].describe()

count    5872.000000
mean       16.246935
std       125.518164
min         0.000000
25%         1.000000
50%         2.000000
75%         6.000000
max      5091.000000
Name: num_comments, dtype: float64

Within this dataset, 75% of posts have 6 or fewer comments. Only 146 posts have 100 or more comments, and only 8 posts have 1000 or more comments. Because the posts with more comments could be exclusively featured in one subreddit or the other, I will not drop any rows, even if they seem like outliers.

### Clean 'Score' Column

In [11]:
subreddits['score'].isna().sum()

0

In [12]:
subreddits['score'].describe()

count     5872.000000
mean        32.774183
std        606.188675
min          0.000000
25%          1.000000
50%          1.000000
75%          1.000000
max      28606.000000
Name: score, dtype: float64

As shown above, 75% of all posts in this data set have a score of 1 or below. Only 81 posts have a score of 100 or more, and only 25 posts have a score of 1000 or higher. Because the high scores could be characteristic of one subreddit, I will not drop any rows based on score. 

### Clean 'Timestamp' Column

In [13]:
subreddits['timestamp'].isna().sum()

0

In [14]:
subreddits['timestamp'].describe()

count           5872
unique           132
top       2020-07-04
freq             219
Name: timestamp, dtype: object

The timestamp data looks okay.

## Basic Feature Engineering

Upon looking at the data, there are some features that I know that I would like to create to without exploring the data. I will create these features here. 

### Engineer a Total Text Column

In [15]:
subreddits['total_text'] = subreddits['title'] + subreddits['selftext']

### Engineer a Post Length (Characters) Feature

In [16]:
subreddits['post_length_char'] = subreddits['total_text'].map(len)

### Engineer a Post Length (Word Count) Feature

In [17]:
subreddits['post_length_words'] = subreddits['total_text'].map(lambda x: len(x.split()))

## Export Clean Data

In [18]:
subreddits.to_csv('../data/subreddits_clean.csv')